In [86]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True, timeout_ms = 0)

Mounted at /content/drive


In [87]:
base_dir = '/content/drive/My Drive/'

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
! pip install datasets
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [74]:
import torch
import pandas as pd
import os
import datetime

from transformers import (BertTokenizer,
                          AutoModelForSequenceClassification,
                          Trainer,
                          TrainingArguments,
                          TrainerCallback)
from datasets import (Dataset,
                      load_dataset,
                      load_metric,
                      load_dataset_builder,
                      ClassLabel,
                      Value,
                      Features)

In [ ]:
BATCH_SIZE = 32

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)


cuda:0


## Model

In [82]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [83]:
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], padding="max_length", truncation=True)

In [85]:
model = AutoModelForSequenceClassification.from_pretrained("an-eve/bert-base-uncased-mnli-2-labels")
model.to(DEVICE)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [88]:
metric = load_metric('glue', "mnli")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [89]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

## Idioms

### Adversarial Definitions
Silver non-entailment, replacing an idiom used in a figuartive context with adversarial definition

In [66]:
url_magpie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/adversarial_definition_ne_magpie.tsv"
url_pie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/adversarial_definition_ne_pie.tsv"
url_semeval = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/adversarial_definition_ne_semeval.tsv"

In [67]:
adv_magpie_df = pd.read_csv(url_magpie, sep='\t', names=['premise','hypothesis', 'label'])
adv_pie_df = pd.read_csv(url_pie, sep='\t', names=['premise','hypothesis', 'label'])
adv_semeval_df = pd.read_csv(url_semeval, sep='\t', names=['premise','hypothesis', 'label'])

In [98]:
adv_magpie_df['label'] = 1
adv_pie_df['label'] = 1
adv_semeval_df['label'] = 1

In [71]:
adv_ne_df = pd.concat([adv_magpie_df, adv_pie_df, adv_semeval_df], ignore_index=True)
print("Adversarial Definitions Dataframe shape: ", adv_ne_df.shape)
adv_ne_df.head()

Adversarial Definitions Dataframe shape:  (6116, 3)


,premise,hypothesis,label
0,"Taking the EG-1 first, the HFS bridge humbucke...","Taking the EG-1 first, the HFS bridge humbucke...",1
1,At the OK Corral — as she preferred to call wh...,At the OK Corral — as she preferred to call wh...,1
2,That is how he ducks out of his responsibiliti...,That is how he ducks out of his responsibiliti...,1
3,"Without his moustache, the little spin magicia...","Without his moustache, the little spin magicia...",1
4,"Apartheid arguments over, the Commonwealth's n...","Apartheid arguments over, the Commonwealth's n...",1


In [72]:
adv_ne_df.drop_duplicates(inplace=True, ignore_index=True)
print("Adversarial Definitions Dataframe shape after considering duplicates: ", adv_ne_df.shape)

Adversarial Definitions Dataframe shape after considering duplicates:  (6035, 3)


In [93]:
adv_ne_dataset = Dataset.from_pandas(adv_ne_df)
print(adv_ne_dataset)
print(adv_ne_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 6035
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [94]:
new_features = adv_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

adv_ne_dataset = adv_ne_dataset.cast(new_features)

print(adv_ne_dataset.features)

Casting the dataset:   0%|          | 0/6035 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [95]:
adv_ne_dataset[:10]

{'premise': ["Taking the EG-1 first, the HFS bridge humbucker delivers a warm, fat sound reminiscent of a vintage PAF, and while the guitar doesn't have the image of a rocker the bridge pickup is almost arrogant in the way it handles an amp at full tilt.",
  "At the OK Corral — as she preferred to call what some of its visitors termed her quim (the phrase struck her as useful after she'd seen the Western)— some men proved themselves and others came to grief, and both kinds had male competitors on their mind and wanted to outgun them.",
  'That is how he ducks out of his responsibilities and will not come clean — Now I shall sit down.',
  'Without his moustache, the little spin magician looked more boyish than ever, prompting the thought that unless he cooled down he might benefit from six of the best from the carpet - slipper.',
  "Apartheid arguments over, the Commonwealth's now seeking a new role spreading democracy, but the assembly lacks stars and so far the Cypriots have stolen th

In [96]:
tokenized_adv_ne_dataset = adv_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6035 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [97]:
args_test = TrainingArguments("Adversarial-Definitions",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_adv_ne_dataset,
    eval_dataset=tokenized_adv_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 2.4292047023773193,
 'eval_accuracy': 0.40314830157415077,
 'eval_runtime': 207.6802,
 'eval_samples_per_second': 29.059,
 'eval_steps_per_second': 0.91}

### Figurative Context
Silver entailment, replacing an idiom used in a figuartive context with definition

In [99]:
url_magpie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/fig_context_magpie_e.tsv"
url_pie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/fig_context_pie_e.tsv"
url_semeval = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/fig_context_semeval_e.tsv"

In [100]:
fig_magpie_df = pd.read_csv(url_magpie, sep='\t', names=['premise','hypothesis', 'label'])
fig_pie_df = pd.read_csv(url_pie, sep='\t', names=['premise','hypothesis', 'label'])
fig_semeval_df = pd.read_csv(url_semeval, sep='\t', names=['premise','hypothesis', 'label'])

In [107]:
fig_magpie_df['label'] = 0
fig_pie_df['label'] = 0
fig_semeval_df['label'] = 0

In [108]:
fig_e_df = pd.concat([fig_magpie_df, fig_pie_df, fig_semeval_df], ignore_index=True)
print("Figurative Context Dataframe shape: ", fig_e_df.shape)
fig_e_df.head()

Figurative Context Dataframe shape:  (16652, 3)


,premise,hypothesis,label
0,"‘ Well, it's a recognised symptom of the condi...","‘ Well, it's a recognised symptom of the condi...",0
1,"‘ Well, it's a recognised symptom of the condi...","‘ Well, it's a recognised symptom of the condi...",0
2,"Taking the EG-1 first, the HFS bridge humbucke...","Taking the EG-1 first, the HFS bridge humbucke...",0
3,The Prime Minister believes that her words str...,The Prime Minister believes that her words eli...,0
4,"He was dressed more casually, and, just as he ...","He was dressed more casually, and, just as he ...",0


In [109]:
fig_e_df.drop_duplicates(inplace=True, ignore_index=True)
print("Figurative Contex Dataframe shape after considering duplicates: ", fig_e_df.shape)

Figurative Contex Dataframe shape after considering duplicates:  (15641, 3)


In [110]:
fig_e_dataset = Dataset.from_pandas(fig_e_df)
print(fig_e_dataset)
print(fig_e_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 15641
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [111]:
new_features = fig_e_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

fig_e_dataset = fig_e_dataset.cast(new_features)

print(fig_e_dataset.features)

Casting the dataset:   0%|          | 0/15641 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [112]:
fig_e_dataset[:10]

{'premise': ["‘ Well, it's a recognised symptom of the condition that an attack can come out of the blue.",
  "‘ Well, it's a recognised symptom of the condition that an attack can come out of the blue.",
  "Taking the EG-1 first, the HFS bridge humbucker delivers a warm, fat sound reminiscent of a vintage PAF, and while the guitar doesn't have the image of a rocker the bridge pickup is almost arrogant in the way it handles an amp at full tilt.",
  'The Prime Minister believes that her words strike a chord with the British public.',
  'He was dressed more casually, and, just as he looked good — and then some — in a business suit, Leith realised that to some women his tall, casually clad self was quite something more.',
  'That is how he ducks out of his responsibilities and will not come clean — Now I shall sit down.',
  'That is how he ducks out of his responsibilities and will not come clean — Now I shall sit down.',
  'STRAIGHT FROM THE SHOULDER',
  'He too (‘ birds of a feather’) h

In [113]:
tokenized_fig_e_dataset = fig_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/15641 [00:00<?, ? examples/s]

In [114]:
args_test = TrainingArguments("Figurative-Context",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_fig_e_dataset,
    eval_dataset=tokenized_fig_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.1207855939865112,
 'eval_accuracy': 0.7255290582443578,
 'eval_runtime': 552.8672,
 'eval_samples_per_second': 28.291,
 'eval_steps_per_second': 0.884}

### Literal Context
Silver non-entailment, replacing an idiom used in a literal context with definition

In [115]:
url_magpie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/lit_context_magpie_ne.tsv"
url_pie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/lit_context_pie_ne.tsv"
url_semeval = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/lit_context_semeval_ne.tsv"

In [116]:
lit_magpie_df = pd.read_csv(url_magpie, sep='\t', names=['premise','hypothesis', 'label'])
lit_pie_df = pd.read_csv(url_pie, sep='\t', names=['premise','hypothesis', 'label'])
lit_semeval_df = pd.read_csv(url_semeval, sep='\t', names=['premise','hypothesis', 'label'])

In [121]:
lit_magpie_df['label'] = 1
lit_pie_df['label'] = 1
lit_semeval_df['label'] = 1

In [123]:
lit_ne_df = pd.concat([lit_magpie_df, lit_pie_df, lit_semeval_df], ignore_index=True)
print("Literal Context Dataframe shape: ", lit_ne_df.shape)
lit_ne_df.head()

Literal Context Dataframe shape:  (886, 3)


,premise,hypothesis,label
0,"One night of winter, with howling winds blowin...","One night of winter, with howling winds blowin...",1
1,Murder in the docks.,Murder under scrutiny.,1
2,Murder in the docks.,Murder under close scrutiny.,1
3,Nobody gets away with murder in this City.’,Nobody does bad things and not be punished in ...,1
4,Nicolaus speculates that this could give ecolo...,Nicolaus speculates that this could give ecolo...,1


In [124]:
lit_ne_df.drop_duplicates(inplace=True, ignore_index=True)
print("Literal Contex Dataframe shape after considering duplicates: ", lit_ne_df.shape)

Literal Contex Dataframe shape after considering duplicates:  (881, 3)


In [125]:
lit_ne_dataset = Dataset.from_pandas(lit_ne_df)
print(lit_ne_dataset)
print(lit_ne_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 881
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [126]:
new_features = lit_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

lit_ne_dataset = lit_ne_dataset.cast(new_features)

print(lit_ne_dataset.features)

Casting the dataset:   0%|          | 0/881 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [127]:
lit_ne_dataset[:10]

{'premise': ['One night of winter, with howling winds blowing and heavy rain lashing the streets, he arrived cold and wet at the Club, went straight to the bar as usual, and was given his usual drink by the bar man, ‘ Johnny’.',
  'Murder in the docks.',
  'Murder in the docks.',
  'Nobody gets away with murder in this City.’',
  'Nicolaus speculates that this could give ecologists a method of controlling predation on eggs of rare birds.',
  'Vehicles for off- and on - road use feature in our special Country Car features.',
  'Half a minute later, as he turned the corner into Oxford Street, he was blown off his feet by the explosion just yards from his cab.',
  'He turned the corner miserably, bumping into Elsie with their fish and chips.',
  "These less direct consequences of man's animality could, Malinowski argued, be understood in the light of their integrative contribution to man's social existence.",
  'Switzerland is famous for six cheeses, four of which belong to the semi - har

In [128]:
tokenized_lit_ne_dataset = lit_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/881 [00:00<?, ? examples/s]

In [130]:
args_test = TrainingArguments("Literal-Context",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_lit_ne_dataset,
    eval_dataset=tokenized_lit_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.1655007600784302,
 'eval_accuracy': 0.6787741203178207,
 'eval_runtime': 27.8063,
 'eval_samples_per_second': 31.683,
 'eval_steps_per_second': 1.007}

### Antonyms

Gold non-entailment, manual replacement of idiomatic definition with antonym

In [137]:
url_ant = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/manual_antonyms_ne.tsv"

In [138]:
ant_df = pd.read_csv(url_ant, sep='\t', names=['premise','hypothesis', 'label'])

In [141]:
ant_df['label'] = 1

In [142]:
print("Antonyms Dataframe shape: ", ant_df.shape)
ant_df.head()

Antonyms Dataframe shape:  (375, 3)


,premise,hypothesis,label
0,How have you weathered the storm ?,How have you failed in getting through the dif...,1
1,He prefers acting with other countries to goin...,He prefers acting with other countries to doin...,1
2,The imagination trembles at some of these idea...,The imagination trembles at some of these idea...,1
3,"Attractive , popular and smart , Veronica hove...","Attractive , popular and smart , Veronica hove...",1
4,"All her insecurities surged painfully as , thr...","All her insecurities surged painfully as , car...",1


In [143]:
ant_ne_dataset = Dataset.from_pandas(ant_df)
print(ant_ne_dataset)
print(ant_ne_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 375
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [144]:
new_features = ant_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

ant_ne_dataset = ant_ne_dataset.cast(new_features)

print(ant_ne_dataset.features)

Casting the dataset:   0%|          | 0/375 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [145]:
ant_ne_dataset[:10]

{'premise': ['How have you weathered the storm ?',
  'He prefers acting with other countries to going it alone .',
  "The imagination trembles at some of these ideas — will a profession 's ruling body really come clean about its members ' income?— but no suggestion , on this subject , should be dismissed out of hand .",
  'Attractive , popular and smart , Veronica hovers in the orbit of the ‘ Heathers’ , a four - girl clique who rule the roost at Westerburgh High .',
  'All her insecurities surged painfully as , throwing caution to the winds , she confronted him furiously , demanding imperiously , ‘ Why exactly have you brought me here , Rune ?',
  "It was pretty interesting, like one of the guys goes native and one of the guys doesn't.",
  '‘ It breaks my heart that his career has been ruined .',
  'For me it is quite out of the question that the Maastricht treaties should be changed again in one way or another .',
  "London is like ‘ iced water , it 's crystal clear and safe’",
  'Yo

In [146]:
tokenized_ant_ne_dataset = ant_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

In [147]:
args_test = TrainingArguments("Antonym",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_ant_ne_dataset,
    eval_dataset=tokenized_ant_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.255462646484375,
 'eval_accuracy': 0.6533333333333333,
 'eval_runtime': 12.0377,
 'eval_samples_per_second': 31.152,
 'eval_steps_per_second': 0.997}

### Definition
Gold entailment, hand written literal definition of idiomatic sentence

In [148]:
url_def = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/manual_e.tsv"

In [149]:
def_df = pd.read_csv(url_def, sep='\t', names=['premise','hypothesis', 'label'])

In [150]:
def_df['label'] = 0

In [151]:
print("Antonyms Dataframe shape: ", def_df.shape)
def_df.head()

Antonyms Dataframe shape:  (528, 3)


,premise,hypothesis,label
0,How have you weathered the storm ?,How have you succeeded in getting through the ...,0
1,"And then , the Daily Telegraph discovered ‘ th...","And then , the Daily Telegraph discovered ‘ th...",0
2,He prefers acting with other countries to goin...,He prefers acting with other countries to doin...,0
3,The imagination trembles at some of these idea...,The imagination trembles at some of these idea...,0
4,"Attractive , popular and smart , Veronica hove...","Attractive , popular and smart , Veronica hove...",0


In [152]:
def_e_dataset = Dataset.from_pandas(def_df)
print(def_e_dataset)
print(def_e_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 528
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [153]:
new_features = def_e_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

def_e_dataset = def_e_dataset.cast(new_features)

print(def_e_dataset.features)

Casting the dataset:   0%|          | 0/528 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [154]:
def_e_dataset[:10]

{'premise': ['How have you weathered the storm ?',
  "And then , the Daily Telegraph discovered ‘ the truth’ : ‘ privatisation of Britain 's water industry … runs against the grain of Mr Delors’ social strategy.’",
  'He prefers acting with other countries to going it alone .',
  "The imagination trembles at some of these ideas — will a profession 's ruling body really come clean about its members ' income?— but no suggestion , on this subject , should be dismissed out of hand .",
  'Attractive , popular and smart , Veronica hovers in the orbit of the ‘ Heathers’ , a four - girl clique who rule the roost at Westerburgh High .',
  'All her insecurities surged painfully as , throwing caution to the winds , she confronted him furiously , demanding imperiously , ‘ Why exactly have you brought me here , Rune ?',
  "It was pretty interesting, like one of the guys goes native and one of the guys doesn't.",
  '‘ It breaks my heart that his career has been ruined .',
  'For me it is quite out o

In [155]:
tokenized_def_e_dataset = def_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/528 [00:00<?, ? examples/s]

In [156]:
args_test = TrainingArguments("Definition",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_def_e_dataset,
    eval_dataset=tokenized_def_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.8807476162910461,
 'eval_accuracy': 0.7746212121212122,
 'eval_runtime': 16.3553,
 'eval_samples_per_second': 32.283,
 'eval_steps_per_second': 1.039}

### Non-entailment Hand
Gold non-entailment, hand written non-entailed sentence from idiom

In [157]:
url_hand = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/manual_ne.tsv"

In [158]:
hand_df = pd.read_csv(url_hand, sep='\t', names=['premise','hypothesis', 'label'])

In [159]:
hand_df['label'] = 1

In [160]:
print("Antonyms Dataframe shape: ", hand_df.shape)
hand_df.head()

Antonyms Dataframe shape:  (254, 3)


,premise,hypothesis,label
0,How have you weathered the storm ?,How have you calmed the storm?,1
1,"And then , the Daily Telegraph discovered ‘ th...","And then , the Daily Telegraph discovered ‘ th...",1
2,"All her insecurities surged painfully as , thr...","All her insecurities surged painfully as , thr...",1
3,"It was pretty interesting, like one of the guy...","It was pretty interesting, like one of the guy...",1
4,‘ It breaks my heart that his career has been ...,‘ It touched my heart that his career has been...,1


In [161]:
hand_ne_dataset = Dataset.from_pandas(hand_df)
print(hand_ne_dataset)
print(hand_ne_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 254
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [162]:
new_features = hand_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

hand_ne_dataset = hand_ne_dataset.cast(new_features)

print(hand_ne_dataset.features)

Casting the dataset:   0%|          | 0/254 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [163]:
hand_ne_dataset[:10]

{'premise': ['How have you weathered the storm ?',
  "And then , the Daily Telegraph discovered ‘ the truth’ : ‘ privatisation of Britain 's water industry … runs against the grain of Mr Delors’ social strategy.’",
  'All her insecurities surged painfully as , throwing caution to the winds , she confronted him furiously , demanding imperiously , ‘ Why exactly have you brought me here , Rune ?',
  "It was pretty interesting, like one of the guys goes native and one of the guys doesn't.",
  '‘ It breaks my heart that his career has been ruined .',
  'For me it is quite out of the question that the Maastricht treaties should be changed again in one way or another .',
  'Only elves and trolls had survived the coming of Man to the discworld ; the elves because they were altogether too clever by half , and the trollen folk because they were at least as good as humans at being nasty , spiteful and greedy .',
  'If the front rabbit sits tight and declines to budge there is no way the others ca

In [164]:
tokenized_hand_ne_dataset = hand_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/254 [00:00<?, ? examples/s]

In [165]:
args_test = TrainingArguments("Hand N-E",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_hand_ne_dataset,
    eval_dataset=tokenized_hand_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 2.7674906253814697,
 'eval_accuracy': 0.28346456692913385,
 'eval_runtime': 7.9017,
 'eval_samples_per_second': 32.145,
 'eval_steps_per_second': 1.012}

## Metaphors